
#### 1. Bronze table 을 생성합니다.
  - 배치로 데이터를 수집하여 Bronze table 을 생성합니다.
  - source file : s3://ddbx-academy/usage-data/korea_tour/20250122/
  - External table 로 생성해 주세요.
  - External location : s3://{my_bucket}/ext_tbl/

|Type|Source Path|수집간격|mode|
|---|---|---|---|
|batch|s3://ddbx-academy/usage-data/korea_tour/{yyyyMMdd}/|매일 07시 00분 실행|append|

- etl_ymdh 컬럼을 추가해 주세요 (timestamp type)


### 스케쥴링을 위한 날짜 생성 

- 한국 시간 적용을위해 +9

In [0]:
from datetime import datetime, timedelta

now = (datetime.now()+ timedelta(hours=9)).strftime('%Y%m%d')

print(now)



### DataFrame 생성

In [0]:
df = spark.read.format('csv').option('header', 'true').load(f's3://ddbx-academy/usage-data/korea_tour/{now}/')


### etl_ymdh column 추가

In [0]:
    from pyspark.sql.functions import lit,current_timestamp

final_df = df.withColumn('etl_ymdh', lit(current_timestamp()))


### 임시 뷰 생성

In [0]:
final_df.createTempView('temp_view')


### Bronze table 생성 

In [0]:
%sql

CREATE TABLE IF NOT EXISTS edu2501.inqyu_park_data_dynamics_io.korea_tour_bronze
USING DELTA 
TBLPROPERTIES ('delta.columnMapping.mode' = 'name')
LOCATION 's3://ddbx-user-bucket/ext_tbl01/'
AS SELECT * FROM temp_view


### INSERT INTO 
- 배치시 적용

❗️해당 노트북에서 초기적재에 사용하면 중복 적재 됩니다.

In [0]:
%sql

INSERT INTO edu2501.inqyu_park_data_dynamics_io.korea_tour_bronze
SELECT * FROM temp_view